In [1]:
import json
with open("./board.json", "r") as f:
    board = json.load(f)
print(board)

[{'id': 1, 'attackerCost': 4, 'defenderCost': 4, 'attackerProb': 0.79, 'defenderProb': 0.69, 'severity': 3, 'risk': 8.82}, {'id': 2, 'attackerCost': 1, 'defenderCost': 3, 'attackerProb': 0.82, 'defenderProb': 0.97, 'severity': 4, 'risk': 1.18}, {'id': 3, 'attackerCost': 5, 'defenderCost': 5, 'attackerProb': 0.5, 'defenderProb': 0.77, 'severity': 4, 'risk': 1.84}, {'id': 4, 'attackerCost': 5, 'defenderCost': 5, 'attackerProb': 0.6, 'defenderProb': 0.77, 'severity': 3, 'risk': 1.84}]


In [2]:
import itertools
def powerset(s):
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1, len(s)+1))

In [3]:
def get_utility(attackerMoves, defenderMoves):
    defenderLabels = set([m["id"] for m in defenderMoves])
    return sum([
        move["severity"]**2 * move["attackerProb"]
        if move["id"] not in defenderLabels else
        move["severity"]**2 * move["attackerProb"] * (1-move["defenderProb"])
        for move in attackerMoves
    ])

In [4]:
def validate_cost(moves, cost_func, max_cost):
    return sum([cost_func(m) for m in moves]) <= max_cost

In [5]:
import numpy as np
all_move_sets = list(powerset(board))
attacker_utility_matrix = np.zeros((len(all_move_sets),len(all_move_sets)))
defender_utility_matrix = np.zeros((len(all_move_sets),len(all_move_sets)))
print(f"Found {len(all_move_sets)} move combinations ({len(all_move_sets) ** 2:,} for both)")

Found 15 move combinations (225 for both)


In [6]:
attacker_cost_known_to_defender = False
attacker_cost_available=10
defender_cost_available=10
for i, attacker_choice in enumerate(all_move_sets):
    for j, defender_choice in enumerate(all_move_sets):
        # if the attacker moveset is invalid
        if not validate_cost(attacker_choice, lambda x: x["attackerCost"], attacker_cost_available):
            # terrible attacker utility
            attacker_utility_matrix[i][j] = -999
            # if the defender moveset is also invalid
            if not validate_cost(defender_choice, lambda x: x["defenderCost"], defender_cost_available):
                # terrible defender utility
                defender_utility_matrix[i][j] = -999
            else:
                # utility is dependent on whether the defender knows the attacker max cost or not
                defender_utility_matrix[i][j] = -999 if attacker_cost_known_to_defender else -get_utility(attacker_choice, defender_choice)
        else:
            # normal attacker utility otherwise
            attacker_utility_matrix[i][j] = get_utility(attacker_choice, defender_choice)
            # if defender moveset is invalid
            if not validate_cost(defender_choice, lambda x: x["defenderCost"], defender_cost_available):
                # terrible defender utility
                defender_utility_matrix[i][j] = -999
            else:
                # normal defender utility otherwise
                defender_utility_matrix[i][j] = -get_utility(attacker_choice, defender_choice)

In [7]:
import scipy.io
scipy.io.savemat("../matlab/utility.mat", {
    "attacker_utility": attacker_utility_matrix.transpose(),
    "defender_utility": defender_utility_matrix,
})